# League of Legends Victory Prediction

### Andrew Italo, Conner Brown                          
CPSC 310 - Spring 2019

* * *

## Introduction

Currently the most played video game in the world, League of Legends is of a class of games called MOBAs, which stands for Multiplayer Online Battle Arena. Essentially, two teams of five players meet on a map divided into 3 distinct lanes in order to fight, capture objectives, kill AI-controlled minions and other players, and ultimately to destroy the other team's base. League of Legends is a complex game with many factors that can swing a match in your team's favor, many of which are hard to express as objective numbers and statistics. However, by looking at those factors that can be expressed in numbers, it is possible to get a fairly accurate picture of which team is doing better at a given time.

This project draws on a dataset containing game information for 7,600 professional matches of League of Legends. Each match contains information such as which teams played against each other, which champions were assigned to each lane, which team won, and a minute-by-minute breakdown of various objectives such as kills, towers destroyed, dragon and baron captures, and the gold differential at each minute.

Using this data, we will attempt to build a classifier that can accurately predict which team will win a match. We will be utilizing TDIDT decision tree generation, combined with a post-pruning algorithm to avoid overfitting. In order to test the comparative accuracy of our pruned decision tree, we will test it against both a non-pruned TDIDT tree and a random forest ensemble classifier to see if pruning the tree had any impact on accuracy, as well as a kNN and a Naive Bayes classifier to determine if other classifiers are more suited to this data than decision trees.

----

## Dataset

The dataset we chose includes virtually every factor about a given match that can be expressed as quantifiable data. Many of these, such as which professional teams faced off or which character each player selected, are attributes that would potentially have a very big impact on which side ended up winning, but are hard to include in a classification algorithm. For example, there are dozens of teams, with new teams being formed each year as some older teams disband. Each team occaisionally switches up their roster, so even the eligible players can change entirely over the years. A player's individual skill with a given character is definitely a huge factor, but player / champion relationships creates too many combinations to have an accurate classifier that considers all possibilities, and would leave a huge amount of unseen instances that wouldn't necessarily be able to be classified. Even just considering what 10 champions are present in a given match is a good indication of relative team strength, but the current roster is sitting at over 150 different characters, many of which were released during the time period that our dataset spans. Not only that, but Riot Games, the company behind League of Legends, releases frequent patches to the game for balancing purposes, and so the strength of each champion will change over time pretty drastically. So, a team composition that was almost guaranteed to win in 2016 might be horribly unoptimized in 2018. Combined with the fact that using team composition as a relative attribute would mean that all future champion releases would constitute unseen data, it would be incredibly difficult and time consuming to create a classifier that uses these attributes, even though they are some of the most important factors that go into the relative starting position of one team over another. 

As such, we have pruned our dataset down to only those attributes that meet several rules:

* **Attribute must be easy to represent in objective, quantifiable measurements**
    * This eliminates factors such as how well a team groups up and communicates, or their overall strategy for winning the match
* **Attribute must be unlikely to change drastically over time due to changes in the game's design or changes to the competitive scene**
    * This eliminates attributes such as champion selection or team roster
* **Attribute must intuitively contribute to a team's win or loss in some meaningful manner**
    * This eliminates attributes such as what year the game was played, or what league it was played in
* **Attribute must not create too many combinations to create a feasible classifier**
    * This eliminates attributes such as the location of a given kill on the map, or the individual kill count of each team member
    * This does not include attributes that can be discretized, such as gold difference
* **Attribute must not be a repeat or different representation of a previously included attribute**
    * This eliminates attributes such as blue team's total gold and red team's total gold, since we have one attribute that measures the difference in gold between the two teams

Here is the final list of attributes included in our data, along with a description of each:

| Attribute | Description |
| --------- | ----------- |
| **gamelength** | The duration in minutes of a given match |
| **bResult** | The winner of a given match <br> 0 = red team won <br> 1 = blue team won |
| **golddiff** | The difference in gold between the two teams <br> positive values indicate blue team is ahead <br> negative values indicate red team is ahead <br> Gold is generated through killing AI-controlled minions, other players, objectives, as well as at a very slow passive rate|
| **bKills / rkills** | The total number of times a player has killed a player of the opposite team <br> Players respawn after being killed, although the death timer increases based on the current time in the game|
| **bTowers / rTowers** | The total number of towers destroyed by each team <br> Towers are stationary structures that damage enemy players when they get too close <br> Towers must be destroyed to push in towards the enemy base <br> In total, each team starts with 11 symmetrically placed towers along the lanes on their side of the map <br> Towers do not respawn when destroyed |
| **bInhibs / rInhibs** | The total number of inhibitors destroyed by each team <br> Inhibitors are stationary structures that enhance your AI-controlled minions when destroyed <br> At least one Inhibitor must be destroyed in order to damage the other team's base <br> In total, each team starts with 3 symmetrically placed Inhibitors within their base <br> Inhibitors do respawn several minutes after being destroyed |
| **bDragons / rDragons** | The total number of Dragons killed by each team <br> The Dragon is a neutral map objective that provides a permanent teamwide buff to the team that kills it <br> There is one Dragon present on the map at a given time, always in the same spot <br> The Dragon does respawn several minutes after being killed |
| **bBarons / rBarons** | The total number of times each team has killed Baron Nashor <br> Baron is an incredibly powerful neutral objective that requires an entire team to fight <br> Once killed, it grants your team a very powerful stat boost that dissappears after a certain time or if you are killed <br> There is one Baron present on the map at a given time, always in the same spot <br> Baron does respawn several minutes after being killed |
| **bHeralds / rHeralds** | Indicates if / when a given team killed the Rift Herald <br> The Herald is a neutral objective that allows the team that kills it to then summon it to their side and help destroy towers quickly <br> There is only one Herald on the map, always in the same spot, which disappears at a certain time in the game <br> The Herald does not respawn after being killed or disappearing, and only one will ever exist in a given game |

Of these attributes, gamelength and bResult are static values that are only available after a game has ended. Gloddiff and kills are the two easiest measures of how well a team is doing at a given point and how strong they are relative to the other team. Towers and Inhibitors must be destroyed to make progress towards the other team's base, and so indicate how well a team is pushing towards ending the game, regardless of their relative strength to the other team. Dragons, Barons, and Rift Heralds are all strategic elements that grant situational power to one side, and although they can have a huge impact on the game, they are not required to win and their impact is dependent on how well a team leverages the advantage gained from capturing them against how well the other team is able to respond. For example, if a team kills Baron Nashor and gains the corresponding buff but does not use it to gain better position on the map or kill the other team, or if the other team is able to respond quickly and kill them before they are able to, then there is very little advantage gained from taking the objective.

Golddiff was originally represented as an array with values for each minute in the game, and all other attributes besides the game length and the game's winner have a timestamp for each time they were captured. As such, we have divided each game into several instances, with each instance representing one minute in the game. Therefore, a game that lasts 30 minutes will be divided into 30 separate instances, with each instance showing the difference in gold at the given minute, as well as the number of kills earned, structures destroyed, and objectives secured by each team up to that point in the game. 

----

## Summary Statistics

In total, our dataset consists of 7620 instances, with each instance describing one professional League of Legends match. Once split by minute, our data size increases to 221,358 instances, each instance matching one minute of one match. When analyzing the data, there are several interesting features that are worth noting.

##### Game Length
Analyzing the distribution of game length is important if we are going to classify our dataset by the minute. We can quickly get some overall stats to describe the data.

| Statistic | Minutes | 
| --------- | ------- |
| Shortest Game | 17 |
| Longest Game | 95 |
| Average Length | 37.013 |

It is also useful to be able to visualize this distribution, as in ***Figure 1.***

<img src="images/length.png">
<center><b><i>Figure 1</i></b></center>

This histogram shows a plot of games that are at least *x* minutes long. As we can see, the graph falls of significantly after about 25-30 minutes, which matches the accepted "standard" length of a game of League of Legends. 

It is important to note how the game mechanics will influence our conclusion based on this distribution. In a game of League, all players start from theoretically "even" ground, with the same level and amount of gold. Kills and objectives taken early will set a player up for the later parts of the game, but there is also more time for the other player to catch up, especially considering it is tough to earn a significant lead early in the game. Combined with the fact that player the time for a player to respawn after being killed increases from about 5 seconds at the start of the game to about 45-50 seconds by its end means that early kills are also eaier to recover from, and so it will be difficult to classify with any significant accuracy in the first minutes of the game. This creates an interesting dichotemy in the late game though; if a team, or even one player, is doing very well in the early and mid game, they can build an advantage in power that is hard to overcome. This can lead to "run-away" games, where it is obvious one team is going to win and it's only a matter of waiting the last 10 minutes it will take for them to do so. However, if the teams are evenly split, the fact that deaths become more and more significant as the game goes on means that a single bad fight can swing the game in a matter of minutes. This effect becomes more and more prominent the longer a game goes on, and so we expect games that look incredibly even up until the last few minutes to also be common in the data. This also poses a challenge to classification, as it is difficult to predict which team will win the game-defining fight without many statistical advantages to analyze. This can even happen when the teams aren't even, and it is entirely possible for a team with a huge lead to make a few small mistakes and suddenly have their opponent close the power gap, putting the two even again. For these reasons, we expect our classifiers to perform best near the end of the game, with unusually long games being less accurate.  


##### Gold as a Predictor
Intuitively, gold is the strongest objective measure of which team is more powerful. Over the course of the game, gold allows you to purchase items, which offer raw stat bonuses and unique effects that make you stronger. Almost every factor in the game has a gold representation; killing minions to push towards the enemy base, killing players, taking objectives, destroying towers, all of these grant a certain amount of gold. Therefore, gold is the only attribute which is included in every possible win condition and strategy. This is a two-way relationship: more gold makes you stronger, and the stronger you are the easier it is to earn gold. This has an interesting effect on the data in that gold differential partway through a match is the best overal attribute for classification prediction, but nearer the end of the game it becomes less of a predictor and more of an indicator. It is almost impossible to win the game with less gold than the other team, since the final steps to win the game would almost always generate a massive amount of gold. This distribution is shown below, where ***Figure 2.*** shows the average gold differential for the winning team.  

<img src="images/golddiff.png">
<center><b><i>Figure 2</i></b></center>

As we can see, the winning team almost always has the gold lead on average. The only exceptions are very long games in which both teams have reached the point at which gold can no longer upgrade your character, and comebacks are possible regardless of gold leads. This means that, if we include gold in our classifiers, we are almost guaranteed to have very accurate predictions, especially at the sweet spot of game lengths between 20-50 minutes, which is where most games tend to end if there are no exceptional circumstances or drawn out stalemates. Because gold is such a good predictor, it tends to overpower other attributes when doing our classification, which makes for boring classification rules. For example, our TDIDT trees would only split on gold if that was an attribute passed into the algorithm. Because of this, we have actually opted not to use gold as a predicting attribute for many of our classifiers, instead trying to predict the outcome based on the other stats we have listed thus far.

##### Side Advantage
League of Legends is marketed as a competitive game, especially at this professional level of play. Like any good competitive game, your win or loss should be based on your skill as an individual and your communication and tactics as a team, rather than random elements. If a game could be decided by a coin toss regardless of player skill, then it wouldn't be popular at any professional level to begin with. Because of this, we would expect that non-interactive aspects of a match should not have an effect on the outcome. League of Legends does a good job at eliminating those randomized elements that detract from skill-based play, and since both teams start off at the same level, the same relative map position, and with the same amount of gold, you would expect that the outcome should be soley decided by factors that occur during a match. Some factors, such as the composition of each team's champions, do have an effect of the game before it starts, since some characters are stronger than others. However, this is also skill and knowledge based, rather than random. Unfortunately, there is one element that players have consistantly complained about being unintuitive and detracting from the skill-based element of the game: side advantage. 

If League of Legends had a perfectly symmetrical map and all things were well balanced, we would expect there to be no advantage to starting on the blue side or the red side, since this is randomly decided at the start of a game and not a decision by either team. However, League of Legends' map uses strategic asymmetry to make the map more fun and promote different competitive moments for each team to prioritize. Overall, the map is 80-90% symmetrical, with only a few differences between starting red or blue. However, due to the camera's fixed angle and the HUD position on screen, as well as minor influencing factors from map layout, this tends not to be the case. In fact, analysis done on complete datasets accross all levels of play found that the blue side actually won close to 52% of games, instead of an even 50/50 split. This may seem small, but considering that over 60 million games of League are played daily, this actually accounts for a huge advantage based on random selection. In professional play, players know how to leverage these advantages more, and so the difference is a bit higher even. From ***Firgure 3.***, we can see that an average of 54% of games are won by the blue side without considering any other factors.

<img src="images/wins.png">
<center><b><i>Figure 3</i></b></center>

This affects our classifiers, because it means that guessing blue more often than red can be a good strategy, especially in the absense of other factors. Especially at the very beginning of a game, when there are no kills, objectives captured, or difference in gold, our classifier is able to get above 50% accuracy simply by guessing blue. However, we believe that this correlation is weak enough that, once other factors start coming in to play, they will drown out the slight imbalance in side advantage, and past about the 5 minute mark starting side should be less of a factor. 

---

## Classifiers

For this project, our new Data Mining topic was adding postpruning to the decision trees generated by TDIDT. To compare the results of these pruned trees, we also classified using non-pruned TDIDT trees, as well as a random forest of TDIDT trees as an ensemble classifier. Then, to see whether decision trees were a good match for this data to begin with, we also classified using kNN and Naive Bayes to see how they compared. 

### TDIDT 
To begin with, we created a decision tree, found the accuracy of the generated rules, and then pruned that tree and found the new accuracy. These were performed side by side in order to use the same tree for both the unpruned and pruned classification, to give a better perpective on how pruning actually affected the accuracy.


In [1]:
import tdidt
tdidt.main()

reading table
converting table numeric
Reducing Range of Several Attributes
Shuffling and Reducing
     Instances:  147572
Stratisfying
Building tree with  98381  instances
['Attribute',
 24,
 ['Value',
  1,
  ['Attribute',
   18,
   ['Value', 1, ['Leaf', 1.0, 0, 0, 0]],
   ['Value', 2, ['Leaf', 1.0, 0, 0, 0]],
   ['Value', 3, ['Leaf', 1.0, 0, 0, 0]],
   ['Value', 4, ['Leaf', 1.0, 0, 0, 0]],
   ['Value', 5, ['Leaf', 1.0, 0, 0, 0]]]],
 ['Value',
  2,
  ['Attribute',
   18,
   ['Value', 1, ['Leaf', 0.0, 0, 0, 0]],
   ['Value',
    2,
    ['Attribute',
     27,
     ['Value', 1, ['Leaf', 1.0, 0, 0, 0]],
     ['Value', 2, ['Leaf', 0.0, 0, 0, 0]],
     ['Value', 3, ['Leaf', 0.0, 0, 0, 0]]]],
   ['Value',
    3,
    ['Attribute',
     27,
     ['Value', 1, ['Leaf', 1.0, 0, 0, 0]],
     ['Value', 2, ['Leaf', 1.0, 0, 0, 0]],
     ['Value', 3, ['Leaf', 1.0, 0, 0, 0]]]],
   ['Value',
    4,
    ['Attribute',
     27,
     ['Value', 1, ['Leaf', 1.0, 0, 0, 0]],
     ['Value', 2, ['Leaf', 1.0, 0, 0

     Pruning
Pruned Tree: 
['Attribute',
 24,
 ['Value', 1, ['Leaf', 1.0, 23802, 16182, 0.4064313127423363]],
 ['Value',
  2,
  ['Attribute',
   18,
   ['Value', 1, ['Leaf', 0.0, 3104, 1065, 0.2602135837511415]],
   ['Value', 2, ['Leaf', 1.0, 12016, 10863, 0.4771137214365818]],
   ['Value', 3, ['Leaf', 1.0, 6949, 1628, 0.1927916798491108]],
   ['Value', 4, ['Leaf', 1.0, 2646, 193, 0.07136370918530092]],
   ['Value', 5, ['Leaf', 1.0, 296, 13, 0.050816969683216584]]]],
 ['Value',
  3,
  ['Attribute',
   18,
   ['Value', 1, ['Leaf', 0.0, 802, 30, 0.04086206941105644]],
   ['Value', 2, ['Leaf', 0.0, 5421, 1319, 0.1991021531065915]],
   ['Value',
    3,
    ['Attribute',
     27,
     ['Value', 1, ['Leaf', 1.0, 1462, 951, 0.401099507562046]],
     ['Value', 2, ['Leaf', 0.0, 1165, 664, 0.3709465281102145]],
     ['Value', 3, ['Leaf', 0.0, 260, 105, 0.30453315637570466]]]],
   ['Value', 4, ['Leaf', 1.0, 1343, 473, 0.2677357835979911]],
   ['Value', 5, ['Leaf', 1.0, 203, 35, 0.1638537810755425

For classification, we used stratified bins to ensure an even distribution of the data, but instead of stratefying based on our classification attribute, we stratified by minutes. This is because, with such a large dataset and such an even split of the two classifiers, we were more worried about having an even distribution of minutes to do our by-the-minute testing. In order to make the data easier to use, we discriminated the attributes based on a series of cutoff values:

| Attribute | Cutoff: 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| --------- | --------- | - | - | - | - | - | - |
| **bKills / rKills** | 10 | 20 | 30 | 40 | 50 | 60 | 75 |
| **bTowers / rTowers** | 0 | 3 | 6 | 9 | 12 | - | - |
| **bInhibs / rInhibs** | 0 | 3 | 6 | 10 | 15 | - | - |
| **bDragons / rDragons** | 3 | 6 | 9 | 12 | 15 | 18 | - |
| **bBarons / rBarons** | 0 | 1 | 2 | - | - | - | - |
| **bHerald / rHerald** | - | - | - | - | - | - | - |


After training it on a train set, we then grouped the test data by minute values, and ran our classifier on each group, getting both the overall accuracy of the classifier and the accuracy at any given minute of a game. ***Figure 4.*** shows the minute by minute accuracy of the unpruned TDIDT tree. After getting these statistics, we then implemented postpruning to the tree we had already generated, and ran the same test data through it to see whether it had an impact on the accuracy of our classifier, as shown in ***Figure 5.***

<img src="images/TDIDT_no_prune.png">
<center><b><i>Figure 4</i></b></center>

<img src="images/TDIDT_pruned.png">
<center><b><i>Figure 5</i></b></center>

As we can see, the two graphs look incredibly similar, with only very subtle differences. We can see that the overall accuracy of the pruned tree is very slightly lower than the unpruned tree on average, usually less than 0.1%. This pruning should help reduce overfitting though, and potentially be more useful on unseen data. It also creates a much smaller tree, and the classification process is noticeably faster on such a large dataset compared to the unpruned tree.

It's also worth noting that our accuracy hovers between 50-60% until about 20 minutes into the game, at which point it spikes up to 70-80%. This means that, as expected, we are much better at predicting the outcome as we reach the middle of most average length games. Our accuracy drops off considerably around 45 minutes, for several reasons. As we've already mentioned, this is usually where one team has already reached their maximum potential power, and the other team then gets a chance to close the gap and catch up. This is also where kills and objectives become the most significant and impactful, so swinging games is much easier and comebacks are more frequent. Finally, this is simply where we have the fewest instances to use for training, meaning that our predictions are often based on only a few instances and so can vary wildly. Overall though, it seems that TDIDT does perform reasonably as a prediction method for most of the game.

### Random Forest

We also chose to compare the accuracy of one decision tree to the accuracy of an ensemble of weak classifier trees. We implemented a random forest to see if we could capture any patterns in the data that a single tree might not. For this, we used the same attributes and discretization methods, except we added one extra. For the purpose of the ensemble classification, we also added the gold differential as a potential attribute to split on. Therefore, our attribute list and discretization cutoffs for the random forest are as follows:

| Attribute | Cutoff: 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 |
| --------- | --------- | - | - | - | - | - | - | - | - | - | - |
| **golddiff** | -8000 | -6000 | -4000 | -2000 | -1000 | 0 | 1000 | 2000 | 4000 | 6000 | 8000 |
| **bKills / rKills** | 10 | 20 | 30 | 40 | 50 | 60 | 75 | - | - | - | - |
| **bTowers / rTowers** | 0 | 3 | 6 | 9 | 12 | - | - | - | - | - | - |
| **bInhibs / rInhibs** | 0 | 3 | 6 | 10 | 15 | - | - | - | - | - | - |
| **bDragons / rDragons** | 3 | 6 | 9 | 12 | 15 | 18 | - | - | - | - | - |
| **bBarons / rBarons** | 0 | 1 | 2 | - | - | - | - | - | - | - | - |
| **bHerald / rHerald** | - | - | - | - | - | - | - | - | - | - | - |

When performing ensemble classification, there are three parameters that must be selected: *N*, the number of trees to generate; *M*, the number of 'best' trees to use for classification, and *F*, the number of randomly selected attributes to split on for each tree. For our forest, we landed on the following values:

| Parameter | Value |
| --------- | ----- |
| N | 25 |
| M | 5 |
| F | 4 |

These values were mainly selected using trial and error, and were selected as the combination that seemed to perform the most consistantly. After choosing these values, we can actually generate our ensemble classifier and use it to predict on instances.

In [2]:
import ensemble
ensemble.main()

reading table
Reducing table size
converting table numeric
Reducing Range of Several Attributes
Shuffling
     Instances:  11067
Generating Random Forest this will take some time
N =  25  M =  5  F =  4
     grouping test set
     running classifier
Sorting accuracies
Minute:  1.0
     Accuracy:  0.42857142857142855
     Correct:  27
     Instances:  63

Minute:  2.0
     Accuracy:  0.41935483870967744
     Correct:  26
     Instances:  62

Minute:  3.0
     Accuracy:  0.5806451612903226
     Correct:  36
     Instances:  62

Minute:  4.0
     Accuracy:  0.4838709677419355
     Correct:  30
     Instances:  62

Minute:  5.0
     Accuracy:  0.5806451612903226
     Correct:  36
     Instances:  62

Minute:  6.0
     Accuracy:  0.6349206349206349
     Correct:  40
     Instances:  63

Minute:  7.0
     Accuracy:  0.6129032258064516
     Correct:  38
     Instances:  62

Minute:  8.0
     Accuracy:  0.6774193548387096
     Correct:  42
     Instances:  62

Minute:  9.0
     Accuracy:  0.69

***Figure 6.*** shows the accuracy accross multiple minutes of the random forest, using the same techniques as the singular TDIDT trees.

<img src="images/randomforestaccuracies.png">
<center><b><i>Figure 6</i></b></center>

For memory and time concerns, we were forced to restrict the number of instances that the random forest predicted on, since it would theoretically take 20 times the runtime of a single decision tree generation if given the same data. However, across multiple tests, the random forest seemed to give better overall accuracies than individual trees, sitting closer to 70-75% accurate over the 